This notebook is used to test out the candidate reward models for the project.

In [ ]:
import pandas as pd
import random
import torch

In [ ]:
# Confirm that the GPU is detected
assert torch.cuda.is_available()
# Get the GPU device name.
device_name = torch.cuda.get_device_name()
n_gpu = torch.cuda.device_count()
print(f"Found device: {device_name}, n_gpu: {n_gpu}")
device = torch.device("cuda")

Found device: Tesla T4, n_gpu: 1


# Dataset

Loading the [Stanford Human Preferences Dataset](https://huggingface.co/datasets/stanfordnlp/SHP).

In [ ]:
!pip install datasets
#!pip install git+https://github.com/yuchenlin/LLM-Blender.git

from datasets import load_dataset
# Load all the data
dataset = load_dataset("stanfordnlp/shp")
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 10.9 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
# Checking domains
domains = [row['domain'] for row in dataset['train']]
# Getting unique values
unique_domains = set(domains)
#print(len(unique_domains))
for domain in unique_domains:
  print(domain)

askphysics_train
changemyview_train
askhr_train
explainlikeimfive_train
asksciencefiction_train
askvet_train
legaladvice_train
askdocs_train
askculinary_train
askhistorians_train
askphilosophy_train
asksocialscience_train
askengineers_train
askcarguys_train
askanthropology_train
askscience_train
askacademia_train
askbaking_train


In [ ]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

# Llama3
[Link](https://huggingface.co/sfairXC/FsfairX-LLaMA3-RM-v0.1) to the model.

In [ ]:
from transformers import AutoTokenizer, pipeline
rm_tokenizer = AutoTokenizer.from_pretrained("sfairXC/FsfairX-LLaMA3-RM-v0.1")

2024-05-05 16:25:06.989201: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 16:25:06.989333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 16:25:07.145523: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
rm_pipe = pipeline(
      "sentiment-analysis",
      model="sfairXC/FsfairX-LLaMA3-RM-v0.1",
      #device="auto",
      device=device,
      tokenizer=rm_tokenizer,
      model_kwargs={"torch_dtype": torch.bfloat16}
  )

config.json:   0%|          | 0.00/849 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/117M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
pipe_kwargs = {
      "return_all_scores": True,
      "function_to_apply": "none",
      "batch_size": 1
  }

In [ ]:
chat = [
   {"role": "user", "content": "Where does the sun rise?"},
   {"role": "assistant", "content": "East"},
   {"role": "user", "content": "Thanks!"},
  ]
test_texts = [rm_tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
pipe_outputs = rm_pipe(test_texts, **pipe_kwargs)
rewards = [output[0]["score"] for output in pipe_outputs]
rewards

/opt/conda/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[-6.5]

In [ ]:
for domain in unique_domains:
  domain_rows = [row for row in dataset['train'] if row['domain'] == domain]
  score = 0
  n = 50 # number of samples
  # Randomly select 5 rows if there are more than 5 rows for this domain
  selected_rows = random.sample(domain_rows, min(n, len(domain_rows)))
  for row in selected_rows:
    post = row['history']
    response_A = row['human_ref_A']
    response_B = row['human_ref_B']
    #input_text = "POST: " + post +" \n\n RESPONSE A: " + response_A + " \n\n RESPONSE B: " + response_B + "\n\n Which response is better? RESPONSE"
    #x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
    #y = model.generate(x, max_new_tokens=1)
    #preference = tokenizer.batch_decode(y, skip_special_tokens=True)

    chat1 = [
    {"role": "user", "content": post},
    {"role": "assistant", "content": response_A},
    {"role": "user", "content": "Thanks!"},
    ]
    chat2 = [
    {"role": "user", "content": post},
    {"role": "assistant", "content": response_B},
    {"role": "user", "content": "Thanks!"},
    ]
    test_texts1 = [rm_tokenizer.apply_chat_template(chat1, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
    pipe_outputs1 = rm_pipe(test_texts1, **pipe_kwargs)
    rewards1 = [output[0]["score"] for output in pipe_outputs1]
    test_texts2 = [rm_tokenizer.apply_chat_template(chat2, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
    pipe_outputs2 = rm_pipe(test_texts2, **pipe_kwargs)
    rewards2 = [output[0]["score"] for output in pipe_outputs2]

    if(rewards1>rewards):
      preference = 'A'
    else:
      preference = 'B'
    if (preference == 'A' and row['labels'] == 1) or (preference == 'B' and row['labels'] == 0):
      score += 1

  print("Score for domain: " + domain + " is: " + str(score) + "/" + str(n))

# SteamXL
[Link](https://huggingface.co/stanfordnlp/SteamSHP-flan-t5-xl) to the model.

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
device = 'cuda'
tokenizer = T5Tokenizer.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl')
model = T5ForConditionalGeneration.from_pretrained('stanfordnlp/SteamSHP-flan-t5-xl').to(device)

In [ ]:
# preference test
input_text = "POST: Instacart gave me 50 pounds of limes instead of 5 pounds... what the hell do I do with 50 pounds of limes? I've already donated a bunch and gave a bunch away. I'm planning on making a bunch of lime-themed cocktails, but... jeez. Ceviche? \n\n RESPONSE A: Lime juice, and zest, then freeze in small quantities.\n\n RESPONSE B: Lime marmalade lol\n\n Which response is better? RESPONSE"
x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
y = model.generate(x, max_new_tokens=1)
tokenizer.batch_decode(y, skip_special_tokens=True)

In [ ]:
# reward score test
input_text = "POST: Instacart gave me 50 pounds of limes instead of 5 pounds... what the hell do I do with 50 pounds of limes? I've already donated a bunch and gave a bunch away. I'm planning on making a bunch of lime-themed cocktails, but... jeez. Ceviche? \n\n RESPONSE A: you should make a lemon meringue pie, and then use the zest from the remaining lemons to create a nice relish.\n\n RESPONSE B: .\n\n Which response is better? RESPONSE"
x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
outputs = model.generate(x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1)
torch.exp(outputs.scores[0][:, 71]) / torch.exp(outputs.scores[0][:,:]).sum(axis=1).item() # index 71 corresponds to the token for 'A'

In [ ]:
# reward score test
input_text = "POST: Instacart gave me 50 pounds of limes instead of 5 pounds... what the hell do I do with 50 pounds of limes? I've already donated a bunch and gave a bunch away. I'm planning on making a bunch of lime-themed cocktails, but... jeez. Ceviche? \n\n RESPONSE A:Lime marmalade lol \n\n RESPONSE B: .\n\n Which response is better? RESPONSE"
x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
outputs = model.generate(x, return_dict_in_generate=True, output_scores=True, max_new_tokens=1)
torch.exp(outputs.scores[0][:, 71]) / torch.exp(outputs.scores[0][:,:]).sum(axis=1).item() # index 71 corresponds to the token for 'A'

In [ ]:
for domain in unique_domains:
    domain_rows = [row for row in dataset['train'] if row['domain'] == domain]
    score = 0
    n = 50 # number of samples
    # Randomly select 5 rows if there are more than 5 rows for this domain
    selected_rows = random.sample(domain_rows, min(n, len(domain_rows)))
    for row in selected_rows:
        post = row['history']
        response_A = row['human_ref_A']
        response_B = row['human_ref_B']
        input_text = "POST: " + post +" \n\n RESPONSE A: " + response_A + " \n\n RESPONSE B: " + response_B + "\n\n Which response is better? RESPONSE"

        x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
        y = model.generate(x, max_new_tokens=1)
        preference = tokenizer.batch_decode(y, skip_special_tokens=True)
        if (preference == ['A'] and row['labels'] == 1) or (preference == ['B'] and row['labels'] == 0):
          score += 1

    print("Score for domain: " + domain + " is: " + str(score) + "/" + str(n))

# Mistral 7B
[Link](https://huggingface.co/weqweasdas/RM-Mistral-7B) to the model.

In [ ]:
from transformers import AutoTokenizer, pipeline
rm_tokenizer = AutoTokenizer.from_pretrained("weqweasdas/RM-Mistral-7B")

In [ ]:
rm_pipe = pipeline(
      "sentiment-analysis",
      model="weqweasdas/RM-Mistral-7B",
      #device="auto",
      device=device,
      tokenizer=rm_tokenizer,
      model_kwargs={"torch_dtype": torch.bfloat16}
  )

In [ ]:
pipe_kwargs = {
      "return_all_scores": True,
      "function_to_apply": "none",
      "batch_size": 1
  }

In [ ]:
#Test example

chat = [
   {"role": "user", "content": "Where does the sun rise?"},
   {"role": "assistant", "content": "Sun rises in the east."},
   {"role": "user", "content": "Thanks!"},
  ]


test_texts = [rm_tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
pipe_outputs = rm_pipe(test_texts, **pipe_kwargs)
rewards = [output[0]["score"] for output in pipe_outputs]
print(rewards)

In [ ]:
for domain in unique_domains:
  domain_rows = [row for row in dataset['train'] if row['domain'] == domain]
  score = 0
  n = 50 # number of samples
  # Randomly select 5 rows if there are more than 5 rows for this domain
  selected_rows = random.sample(domain_rows, min(n, len(domain_rows)))
  for row in selected_rows:
    post = row['history']
    response_A = row['human_ref_A']
    response_B = row['human_ref_B']
    #input_text = "POST: " + post +" \n\n RESPONSE A: " + response_A + " \n\n RESPONSE B: " + response_B + "\n\n Which response is better? RESPONSE"
    #x = tokenizer([input_text], return_tensors='pt').input_ids.to(device)
    #y = model.generate(x, max_new_tokens=1)
    #preference = tokenizer.batch_decode(y, skip_special_tokens=True)

    chat1 = [
    {"role": "user", "content": post},
    {"role": "assistant", "content": response_A},
    {"role": "user", "content": "Thanks!"}
    ]
    chat2 = [
    {"role": "user", "content": post},
    {"role": "assistant", "content": response_B},
    {"role": "user", "content": "Thanks!"}
    ]
    test_texts1 = [rm_tokenizer.apply_chat_template(chat1, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
    pipe_outputs1 = rm_pipe(test_texts1, **pipe_kwargs)
    rewards1 = [output[0]["score"] for output in pipe_outputs1]
    test_texts2 = [rm_tokenizer.apply_chat_template(chat2, tokenize=False, add_generation_prompt=False).replace(rm_tokenizer.bos_token, "")]
    pipe_outputs2 = rm_pipe(test_texts2, **pipe_kwargs)
    rewards2 = [output[0]["score"] for output in pipe_outputs2]

    if(rewards1>rewards):
      preference = 'A'
    else:
      preference = 'B'
    if (preference == 'A' and row['labels'] == 1) or (preference == 'B' and row['labels'] == 0):
      score += 1

  print("Score for domain: " + domain + " is: " + str(score) + "/" + str(n))

# DeBERTa
[Link](https://huggingface.co/OpenAssistant/reward-model-deberta-v3-large-v2) to the model.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
reward_name = "OpenAssistant/reward-model-deberta-v3-large-v2"

In [ ]:
rank_model, tokenizer = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/993 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/455 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:
# test cell
question = "Filling for cupcakes? I have an order for chocolate cupcakes with a variety of fillings but the customer doesn't know what to ask for other than raspberry. Also can't have peanut butter or caramel. Help!"
answer = "Vanilla buttercream, butter-rum, chocolate-raspberry, chocolate-expresso, chocolate-mint, chocolate-cognac, chocolate cherry, or plain cherry or strawberry or raspberry"
inputs = tokenizer(question, answer, return_tensors='pt')
score = rank_model(**inputs).logits[0].cpu().detach()
print(score)

In [ ]:
for domain in unique_domains:
  domain_rows = [row for row in dataset['train'] if row['domain'] == domain]
  score = 0
  n = 50 # number of samples
  # Randomly select 5 rows if there are more than 5 rows for this domain
  selected_rows = random.sample(domain_rows, min(n, len(domain_rows)))
  for row in selected_rows:
    question = row['history']
    answer_1 = row['human_ref_A']
    answer_2 = row['human_ref_B']
    inputs_1 = tokenizer(question, answer_1, return_tensors='pt')
    inputs_2 = tokenizer(question, answer_1, return_tensors='pt')

    score_1 = rank_model(**inputs_1).logits[0].cpu().detach()
    score_2 = rank_model(**inputs_2).logits[0].cpu().detach()


    if(score_1>score_2):
      preference = 'A'
    else:
      preference = 'B'

    if (preference == 'A' and row['labels'] == 1) or (preference == 'B' and row['labels'] == 0):
      score += 1

  print("Score for domain: " + domain + " is: " + str(score) + "/" + str(n))